In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
# ensure cleaning.py is in the same directory
from cleaning import clean_text
from nltk import word_tokenize
from nltk.util import ngrams  
from heapq import nlargest
import collections

In [56]:
tweets_data = pd.read_csv("more-tweets-extracted.csv")

# Hashtags were in string form of list. converting them to python lists..
import ast
tweets_data.hashtags = [ast.literal_eval(x) for x in tweets_data.hashtags]

tweets_data

,user_id,screen_name,tweet_id,tweet_url,timestamp,text,hashtags,has_media,img_urls,video_url,user_location,processed_text,label
0,1139435031204548608,zkwnsaari,1262868325803024389,/zkwnsaari/status/1262868325803024389,2020-05-19 22:10:46,"Cornering men with the phrase ""sexual harassme...",[],0.0,[],NaN,"Perak, Malaysia",corner men phrase sexual harassment abuse power,NaN
1,1262662724628238338,anuglywoman1,1262856464651243525,/anuglywoman1/status/1262856464651243525,2020-05-19 21:23:38,Sexual harassment..is to make someone fucked u...,[],0.0,[],NaN,Pain,sexual harassment make someone fuck mentally,NaN
2,15199808,Vidyut,1262838343404044296,/Vidyut/status/1262838343404044296,2020-05-19 20:11:38,Fellow asking about abuse women face online/st...,[],0.0,[],NaN,India,fellow asking abuse woman face online street h...,NO_DV
3,257394747,PramodChturvedi,1262824245517672449,/PramodChturvedi/status/1262824245517672449,2020-05-19 19:15:36,Case registered against employee of private co...,"[AndhraPradesh, Krishna]",0.0,[],NaN,"Hyderabad, New Delhi, Ballia",case register employee private company sexual ...,NO_DV
4,2247560024,FeminismInIndia,1262816952164196355,/FeminismInIndia/status/1262816952164196355,2020-05-19 18:46:38,Institutional Failures & The Increasing Relian...,[],0.0,[],NaN,India,institutional failure increase reliance social...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1691,857761954018058241,satpal13414,1212326306739703808,/satpal13414/status/1212326306739703808,2020-01-01 10:54:50,#स्वर्ण_युग\nGolden Time Is Coming\nThe other ...,[स],1.0,['https://pbs.twimg.com/media/ENMMVmYU0AAim4b....,NaN,"Rewari, India",स्वर्ण युग golden time coming woman girl would...,DV
1692,1156956207640244224,Mohit__solanki,1212256040089075712,/Mohit__solanki/status/1212256040089075712,2020-01-01 06:15:37,Golden Time Is Coming\nThe other woman and gir...,[स],0.0,[],NaN,"Firozpur, India",golden time coming woman girl would see mother...,DV
1693,838337330322751488,BRAJBHANDAS1234,1212234940441493505,/BRAJBHANDAS1234/status/1212234940441493505,2020-01-01 04:51:46,#स्वर्ण_युग#स्वर्ण_युग\nGolden Time Is Coming\...,"[स, स]",1.0,['https://pbs.twimg.com/media/ENK5O_DUwAEnmUY....,NaN,"Gandhidham, India",स्वर्ण युग स्वर्ण युग golden time coming woman...,DV
1694,972168388490379264,NiteshP82110245,1212212166549331969,/NiteshP82110245/status/1212212166549331969,2020-01-01 03:21:16,#HeavenOnEarth_By_SaintRampalJi\n Time Is Comi...,[HeavenOnEarth_By_SaintRampalJi],1.0,['https://pbs.twimg.com/media/ENKkdxCVAAAxcxF....,NaN,"मध्य प्रदेश, भारत",heavenonearth saintrampalji time coming woman ...,DV


In [6]:
# Making list of all hashtags in corpus
hashtags = []
for ht in tweets_data.hashtags:
    if len(ht):
        hashtags += ht

# Most common hashtags
Counter(hashtags).most_common(100)

[('BanTikTokInIndia', 187),
 ('BanTiktok', 175),
 ('IslamicLawForRapists', 73),
 ('BanTikToklnlndia', 54),
 ('tiktokexposed', 28),
 ('FaizalSiddiqui', 21),
 ('tiktokban', 16),
 ('TikTok', 15),
 ('MeToo', 13),
 ('tiktokdown', 13),
 ('BanTikTokApp', 11),
 ('NirbhayaCase', 9),
 ('BanTikTokIndia', 8),
 ('sexual', 8),
 ('rape', 8),
 ('प', 7),
 ('lockdown', 7),
 ('carryminati', 6),
 ('harassment', 6),
 ('BanTikTokInIndiahttps', 6),
 ('India', 6),
 ('justiceforcarry', 6),
 ('boyslockeroom', 6),
 ('SriLanka', 6),
 ('GenerationEquality', 6),
 ('Constitution_Of_GodKabir', 6),
 ('tiktok', 6),
 ('tiktokindia', 5),
 ('BanTikTok', 5),
 ('boyslockerroom', 5),
 ('TwitterTrolls', 5),
 ('COVID19', 5),
 ('AuratMarch', 5),
 ('InternationalWomensDay', 5),
 ('domesticviolence', 5),
 ('GargiCollege', 5),
 ('BanTiktokhttps', 5),
 ('mondaythoughts', 5),
 ('20MFORCARRY', 5),
 ('PakistanisAgainstChildAbuse', 5),
 ('JusticeForChildren', 5),
 ('news', 5),
 ('childrights', 5),
 ('stopchildsexualabuse', 5),
 ('AsimR

In [57]:
# Cleaning text
clean_text(tweets_data,'text')

Cleaning text and adding column 'processed_text'
DONE! 



In [8]:
all_text = ' '.join(str(word) for word in tweets_data.processed_text.values) 
tokens = word_tokenize(all_text)
    
unigrams = ngrams(tokens, 1)
unigrams = collections.Counter(unigrams).most_common(15)
unigrams

[(('sexual',), 1215),
 (('abuse',), 617),
 (('harassment',), 462),
 (('rape',), 449),
 (('child',), 389),
 (('woman',), 352),
 (('violence',), 319),
 (('content',), 258),
 (('promote',), 236),
 (('case',), 226),
 (('pic',), 214),
 (('india',), 202),
 (('assault',), 196),
 (('tiktok',), 195),
 (('bantiktok',), 194)]

In [9]:
bigrams = ngrams(tokens, 2)
bigrams = collections.Counter(bigrams).most_common(15)
bigrams

[(('sexual', 'harassment'), 282),
 (('sexual', 'abuse'), 259),
 (('sexual', 'content'), 205),
 (('acid', 'attack'), 167),
 (('sexual', 'assault'), 157),
 (('child', 'sexual'), 120),
 (('promote', 'acid'), 117),
 (('rape', 'threat'), 94),
 (('sexual', 'violence'), 78),
 (('domestic', 'violence'), 77),
 (('religion', 'community'), 67),
 (('community', 'abuse'), 67),
 (('youth', 'india'), 63),
 (('child', 'abuse'), 53),
 (('attack', 'pic'), 53)]

### Approach 1 - Using LDA for discovering topics

In [21]:
vectorizer = TfidfVectorizer(max_features = 5000, ngram_range=(1,2))

# apply transformation
tf = vectorizer.fit_transform(tweets_data['processed_text']).toarray()
print(f'Shape of transformed matrix : {tf.shape}')
      
# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()

Shape of transformed matrix : (1696, 5000)


In [22]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)] = ['{}'.format(feature_names[i]) 
                                                     for i in topic.argsort()[:-no_top_words - 1:-1]]

    return pd.DataFrame(topic_dict)

In [23]:
# Feel free to experiment different numbers of topics
number_of_topics = 2

# Using LDA for finding relatable topics
model = LatentDirichletAllocation(n_components = number_of_topics, random_state = 42)
model.fit(tf)

no_top_words = 20
display_topics(model, tf_feature_names, no_top_words)

,Topic 0 words,Topic 1 words
0,sexual,sexual
1,pic,abuse
2,content,child
3,promote,sexual abuse
4,bantiktokinindia,case
5,sexual content,assault
6,harassment,violence
7,tiktok,woman
8,bantiktok,sexual assault
9,attack,harassment


Topic 0 words are generally included in awareness posts or ban tiktok posts.

Topic 1 words are headlines or real victim stories.

But this doesn't guarantee a clear-cut classification.

## Approach 2 - Using Keywords to label some broad topics

In [55]:
# Function to filter data - returns pd dataframe containing posts related to DV
# NOTE : processed_text is a column which contains clean text (lower case, lemmatized, stopwords remooved etc.)

def filter_df(df):
    
    # creating a list of selected tweets
    selected = []
    
    for i in range(len(df)):        
      
        # Should contain words in 'AND' combination from the two sets
        if any(word in df.processed_text[i] for word in ['husband','wife','daughter','father','relative','mother', \
                                                         'sister','uncle','grandfather','neighbour','parent','child', \
                                                         'cousin','inlaw','in-law','boyfriend','marital','domestic', \
                                                         'partner','family','maid','housemaid','gay','ex','liquor']) \
        \
        and any(word in df.processed_text[i] for word in ['abuse','violence','domestic','sexual','harass','assault','bitch', \
                                                          'torture','rape','beat','dowry','porn','acidattack','lockdown', \
                                                          'molest','metoo','tiktok','fuck','mental','physical','threat', \
                                                          'blackmail','body','opress','force','bruise','slut','scar', \
                                                          'misogyny','toxic','cheat','verbal','helpline','aware','survey', \
                                                          'initiative','pandemic','commission','work','tiktok','bantiktok', \
                                                          'content','tik','tok','action']) \
        \
        and any(word in df.processed_text[i] for word in ['harass','sexual','physical','rape','shame','abuse','case']):
            selected.append(df.tweet_id[i])
                
    # A dataframe containing selected tweets/posts
    df_filtered = df.loc[df['tweet_id'].isin(selected)]
    
    # Delete duplicates on the basis of same processed text
    df_filtered.drop_duplicates(subset = "processed_text", inplace=True)
    
    # Reset the indices
    df_filtered.reset_index(drop=True, inplace=True)
    print('Now shape of data = ',df_filtered.shape)
    
    return df_filtered

In [59]:
def label_data(df):
    
    # Add a column for labels
    df['label'] = ['']*len(df)
    
    for i in range(len(df)):  
    
        # Find all posts related to domestic violence    
        if any(word in df.processed_text[i] for word in ['husband','wife','daughter','father','relative','mother', \
                                                         'sister','uncle','grandfather','neighbour','parent','child', \
                                                         'cousin','inlaw','in-law','boyfriend','marital','domestic', \
                                                         'partner','family','maid','housemaid','gay','ex','liquor']) \
        and df.label[i] == '':
            df.label[i] = 'DV'

        # Find all posts related to awareness, opinions, not DV
        elif any(word in df.processed_text[i] for word in ['prevent','law','helpline','aware','survey','webinar', \
                                                           'initiative','pandemic','commission','guide', 'article', \
                                                           'responsibility','delete','work','tiktok','bantiktok', \
                                                            'content','tik','tok','action','equality','equal','employee', \
                                                            'company','right','movie','feminism','film','online']) \
        and df.label[i] == '':
            df.label[i] = 'NO_DV'

In [63]:
tweets_data = filter_df(tweets_data)

Now shape of data =  (1423, 13)


In [65]:
label_data(tweets_data)

C:\Users\welcome\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
# Saving the file

tweets_data.to_csv('more-tweets-extracted.csv', index=False)

In [45]:
# Delhi data
data1 = pd.read_csv('DomesticViolence_Twitter_Topstates/domestic_delhi.csv')
data1 = data1.dropna(subset=['text']).reset_index(drop=True)
data1

,username,text,retweets,favorites,replies,tweet_id,user_id,date,hashtags,mentions,tweet_location
0,KatlegoMkalipi,cause it slaps,0,4,1,1263258032735166466,937333317837492224,2020-05-20 23:59:19+00:00,NaN,NaN,Delhi
1,HDalton17,Pia Mia - do it again also slaps,0,2,0,1263257643025498112,1146347241268862977,2020-05-20 23:57:46+00:00,NaN,NaN,Delhi
2,rebaonemomphi,But the back tracks of #bloodandwaternetflix r...,0,0,0,1263257536150601730,968766255514648576,2020-05-20 23:57:21+00:00,#bloodandwaternetflix,NaN,Delhi
3,sgowsmith1988,This new Jeff Rosenstock slaps. He deserves to...,0,1,0,1263257323742478337,1333490503,2020-05-20 23:56:30+00:00,NaN,NaN,Delhi
4,Thandi_263,Imagine Oluwa on this....It slaps already with...,0,2,1,1263257247217483777,938071259530874885,2020-05-20 23:56:12+00:00,NaN,NaN,Delhi
...,...,...,...,...,...,...,...,...,...,...,...
45304,wongcwj,I'm afraid Australia is the dog of PRC. Refer ...,0,4,0,1263093628601163778,59716003,2020-05-20 13:06:02+00:00,NaN,NaN,Delhi
45305,BJP4Bengal,Really? West Bengal won't tolerate such CRUEL ...,288,647,66,1263093590399451136,2984417534,2020-05-20 13:05:53+00:00,NaN,NaN,Delhi
45306,TanwarVir,"Open public transport Train, Bus, Air, rest pe...",0,0,0,1263093524477562880,3017429790,2020-05-20 13:05:38+00:00,NaN,NaN,Delhi
45307,sanaejaz2,"This must find an end, every single silencing,...",33,56,3,1263093365698166790,604259624,2020-05-20 13:05:00+00:00,#StopThreateningSaidAlam,NaN,Delhi


In [46]:
# Cleaning text
clean_text(data1,'text')

Cleaning text and adding column 'processed_text'
DONE! 



In [52]:
df1 = filter_df(data1)
df1

Now shape of data =  (1002, 12)


C:\Users\welcome\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,username,text,retweets,favorites,replies,tweet_id,user_id,date,hashtags,mentions,tweet_location,processed_text
0,mehmoood7,She looks like the mother of gul bhukar and ma...,0,2,0,1263205589896830977,1241650889133342721,2020-05-20 20:30:56+00:00,NaN,NaN,Delhi,look like mother gul bhukar marvi sardard abus...
1,mehmoood7,She looks like the mother of gul bhukar and ma...,0,1,0,1263155850702655488,1241650889133342721,2020-05-20 17:13:17+00:00,NaN,NaN,Delhi,look like mother gul bhukar marvi sardard abus...
2,uchchil,Did u dare to expose d bus sham by ur fav @pri...,0,0,0,1263096605013610501,301506181,2020-05-20 13:17:52+00:00,NaN,@priyankagandhi @INCIndia,Delhi,u dare expose bus sham ur fav priyankagandhi s...
3,Utkarsh_zed,Indian politics going beyond all the limits. T...,0,0,0,1263062066149146624,824319518277300224,2020-05-20 11:00:37+00:00,#PankajPunia #Congress,@INCIndia,Delhi,indian politics going beyond limit illiteracy ...
4,arundd2000,@Arnab Congratulations ! Sonia camp got a big ...,0,0,0,1263029930322952192,2937241051,2020-05-20 08:52:56+00:00,NaN,@Arnab5222,Delhi,arnab5222 congratulations sonia camp get big s...
...,...,...,...,...,...,...,...,...,...,...,...,...
997,livewire,"""All I have with me now is a lot of deep-roote...",1,0,0,1263054358897254403,974540020420788224,2020-05-20 10:30:00+00:00,#MeToo #MeToo #sexualabuse,NaN,Delhi,lot deep root anger entire system world pepper...
998,Kohliprince25,"Scientist of Japan, Professor Venz has reached...",67,20,2,1262971123362193408,1252603590881001477,2020-05-20 04:59:15+00:00,#AdoptVegetarianDiet,@derasachasauda,Delhi,scientist japan professor venz reach conclusio...
999,r_sakla,Finally people understood this app is just pie...,0,0,0,1262958757782474753,3922154904,2020-05-20 04:10:07+00:00,#BanTiktok #tiktokbanindia #tiktokban,NaN,Delhi,finally people understand app piece scrap prom...
1000,htvpk,Important Sexual Health Questions You Are Afra...,0,0,0,1263178963016396802,1412168383,2020-05-20 18:45:08+00:00,NaN,NaN,Delhi,important sexual health question afraid ask htv


In [66]:
label_data(df1)

C:\Users\welcome\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\welcome\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\welcome\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [28]:
all_text = ' '.join(str(word) for word in data1.processed_text.values) 
tokens = word_tokenize(all_text)
    
bigrams = ngrams(tokens, 2)
bigrams = collections.Counter(bigrams).most_common(15)
bigrams

[(('domestic', 'violence'), 437),
 (('piece', 'shit'), 401),
 (('covid', '19'), 360),
 (('gon', 'na'), 315),
 (('death', 'threat'), 307),
 (('کرنل', 'کی'), 278),
 (('کی', 'بیوی'), 276),
 (('fake', 'profile'), 266),
 (('acid', 'attack'), 241),
 (('wan', 'na'), 224),
 (('eat', 'shit'), 221),
 (('mental', 'health'), 216),
 (('son', 'bitch'), 216),
 (('victim', 'card'), 207),
 (('can', 'not'), 193)]